# Ejercicios Resueltos
# Esteban Matias Cancino

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.11252759, 0.09561582, 0.17267024, ..., 0.09162675, 0.1121114 ,
       0.03334953])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.49040531, 0.48118373, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 8754,  6552, 10613, ...,  6988,  6980,  9520])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
mostsim

array([ 6552, 10613,  3616,  8726,  3902])

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.religion.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.religion.misc
talk.religion.misc
talk.religion.misc
talk.politics.mideast
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**



# Importo las librerias

In [2]:
# importo numpy para operaciones numéricas y manejo eficiente de arrays
import numpy as np

# importo CountVectorizer para transformar texto en matriz de conteo
# e importo TfidfVectorizer para transformar texto en matriz TF-IDF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# importo cosine_similarity para calcular la similitud coseno entre vectores
from sklearn.metrics.pairwise import cosine_similarity

# importo MultinomialNB y ComplementNB, clasificadores Naive Bayes adecuados para texto/conteos
from sklearn.naive_bayes import MultinomialNB, ComplementNB

# importo f1_score para evaluar el rendimiento del modelo combinando precisión y recall en una métrica
from sklearn.metrics import f1_score

# importo fetch_20newsgroups para descargar/cargar el dataset de 20 categorías de noticias
from sklearn.datasets import fetch_20newsgroups

# Carga de datos


In [3]:
# descargo el conjunto de entrenamiento del dataset "20 newsgroups" indicando subset='train'
# elimino headers, footers y quotes para quedarme solo con el cuerpo del mensaje
# obtengo un objeto tipo Bunch que contiene .data (lista de textos), .target (etiquetas numéricas) y .target_names (nombres de las categorías)
train_group = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# descargo el conjunto de prueba (subset='test') con la misma limpieza de headers, footers y quotes
# el objeto resultante también incluye .data, .target y .target_names para evaluación del modelo
test_group = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Vectorización


In [4]:
# creo un TfidfVectorizer para transformar los documentos en vectores TF-IDF (pondera frecuencia por importancia)
# configuro stop_words='english' para eliminar palabras vacías comunes en inglés y reducir ruido en la representación
Tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [5]:
# ajusto el Tfidf_vectorizer con los textos de entrenamiento y transformo esos textos en una matriz TF-IDF
# obtengo una matriz dispersa de forma (n_muestras, n_características) donde cada fila representa un documento
X_train = Tfidf_vectorizer.fit_transform(train_group.data)

# transformo los textos de test usando el vectorizador ya ajustado (no vuelvo a ajustar para conservar el mismo vocabulario)
# la salida es otra matriz TF-IDF dispersa compatible con X_train para evaluación
X_test = Tfidf_vectorizer.transform(test_group.data)

# extraigo las etiquetas numéricas (clases) del conjunto de entrenamiento
y_train = train_group.target

# extraigo las etiquetas numéricas (clases) del conjunto de prueba para comparar predicciones posteriormente
y_test = test_group.target

In [6]:
# muestro el tipo de objeto de X_train
print(type(X_train))

# muestro la forma de la matriz: (n_muestras, n_características)
print(f'shape: {X_train.shape}')

# imprimo la cantidad de documentos (número de filas de la matriz)
print(f'cantidad de documentos: {X_train.shape[0]}')

# imprimo el tamaño del vocabulario / dimensionalidad de los vectores (número de columnas)
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101322)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101322


In [7]:
# muestro el tipo de objeto de X_test
print(type(X_test))

# muestro la forma de la matriz de test: (n_muestras, n_características)
print(f'shape: {X_test.shape}')

# imprimo la cantidad de documentos en el conjunto de test (número de filas)
print(f'cantidad de documentos: {X_test.shape[0]}')

# imprimo el tamaño del vocabulario / dimensionalidad de los vectores en test (número de columnas)
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_test.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (7532, 101322)
cantidad de documentos: 7532
tamaño del vocabulario (dimensionalidad de los vectores): 101322


# Similaridad de documentos


In [8]:
# fijo la semilla de numpy en 42 para que las selecciones aleatorias sean reproducibles
np.random.seed(42)

# selecciono 5 índices únicos de documentos del conjunto de entrenamiento
# X_train.shape[0] es el número total de documentos; size=5 pide 5 índices; replace=False evita índices repetidos
random_docs = np.random.choice(X_train.shape[0], size=5, replace=False)

# imprimo los índices de los documentos seleccionados
print(f"Documentos seleccionados: {random_docs}")

Documentos seleccionados: [7492 3546 5582 4793 3813]


In [9]:
# itero sobre los índices de los documentos seleccionados aleatoriamente
for idx in random_docs:
  # imprimo un separador visual para distinguir bloques
  print("=" * 100)
  # indico el índice del documento que se está mostrando
  print(f"Documento original (índice {idx}):")
  # recupero e imprimo el nombre de la clase usando el array de targets y target_names
  print("Clase:", train_group.target_names[y_train[idx]])
  # muestro una vista previa de los primeros 500 caracteres del documento para inspección rápida
  print(f'Preview: {train_group.data[idx][:500]} ...')
  # imprimo otro separador para separar la previsualización de la sección de similares
  print("=" * 100)

  # calculo la similitud coseno entre el documento actual (fila idx) y todos los documentos de entrenamiento
  # el resultado es una matriz 1xN; con [0] obtengo el vector 1D de similaridades
  cossim = cosine_similarity(X_train[idx], X_train)[0]

  # ordeno los índices por similaridad ascendente con np.argsort, invierto el orden para tener descendente
  # [1:6] excluye el propio documento (posición 0) y toma los 5 documentos más similares
  mostsim = np.argsort(cossim)[::-1][1:6] # Obtener los 5 documentos más similares al actual

  # indico que voy a listar los documentos más similares
  print("5 documentos más similares:")
  # itero sobre los índices de los documentos más similares y su posición en la lista
  for i, sim_idx in enumerate(mostsim):
    # muestro un salto de línea y el número de similar (1..5), el índice del documento y la similaridad formateada
    print(f"\nSimilar {i+1} (índice {sim_idx}, similaridad {cossim[sim_idx]:.4f}):")
    # recupero e imprimo la clase del documento similar
    print("Clase:", train_group.target_names[y_train[sim_idx]])
    # muestro una previsualización del documento similar (primeros 500 caracteres)
    print(f'Preview: {train_group.data[sim_idx][:500]} ...')

Documento original (índice 7492):
Clase: comp.sys.mac.hardware
Preview: Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...
5 documentos más similares:

Similar 1 (índice 10935, similaridad 0.7025):
Clase: comp.sys.mac.hardware
Preview: Hey everybody:

   I want to buy a mac and I want to get a good price...who doesn't?  So,
could anyone out there who has found a really good deal on a Centris 650
send me the price.  I don't want to know where, unless it is mail order or
areound cleveland, Ohio.  Also, should I buy now or wait for the Power PC.

Thanks.
BoB
reply via post or e-mail at rrn@po.cwru.edu
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to bangi ...

Similar 2 (índice 7258,

# Primer Documento:


Para el documento 7492 (comp.sys.mac.hardware, consulta sobre sistemas Mac), la similitud coseno identifica correctamente documentos similares en la misma categoría, como otro post del mismo usuario sobre compras de Centris 650 (similaridad 0.7025), lo que tiene sentido por el contexto de hardware Apple y firmas idénticas. Sin embargo, algunos similares divergen hacia hardware IBM debido a brevedad y palabras comunes como "BoB", y la ultima pasa directamente a una clase distinta sin similitud aparente, destacando limitaciones en textos cortos, aunque TF-IDF captura bien temas relacionados.

# Segundo Documento:


En el documento 3546 (comp.os.ms-windows.misc, sobre backups en Windows), las similitudes se alinean lógicamente con discusiones técnicas de hardware PC, como DMA y controladores (e.g., similaridad 0.1996 con un post sobre busmastering), reflejando coherencia temática en categorías de sistemas IBM. Esto valida la efectividad de TF-IDF para vincular contenidos técnicos, aunque las similitudes son moderadas por la especificidad del tema.

# Tercer documento:


El documento 5582 (misc.forsale, venta de componentes como drives y motherboards), muestra alta similitud con otras ofertas de hardware (e.g., 0.4901 con un post buscando motherboards 286), coherente con la etiqueta de ventas. Incluye cruces con gráficos por menciones técnicas, confirmando que la vectorización captura patrones de comercio y specs.

# Cuarto Documento:


Para el 4793 (talk.politics.guns, política de armas en parques nacionales), las similitudes son más débiles (e.g., 0.1767 con un post breve sobre hardware), luego cruza nuevamente otras categorias de politicas, tocando temas como armas, naturaleza u omnipotencia en ateísmo. Esto indica que TF-IDF detecta temas colaterales como riesgos, aunque la ambigüedad reduce precisión, sugiriendo necesidad de embeddings más avanzados para contextos políticos.

# Quinto Documento:


El documento 3813 (rec.sport.hockey, temas generales), se vincula bien con otros posts de hockey, como el primer similar 0.1405 que trata temas muy similares, o almenos la forma textual es similar, confirmando relevancia temática en deportes. Similitudes menores con temas variados muestran límites en textos cortos, pero generales, TF-IDF valida similitudes basadas en contenido y etiquetas, enfatizando su utilidad para clustering temático.



**2**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [10]:
# transpongo la matriz TF-IDF para cambiar la orientación: ahora filas = términos, columnas = documentos
# la nueva forma será (n_características, n_muestras) — es decir (tamaño del vocabulario, cantidad de documentos)
X_terms = X_train.T

In [11]:
# elijo las 5 palabras manualmente
palabras = ['computer', 'god', 'car', 'space', 'politics']

In [12]:
# dado 'palabras' (lista de términos), obtengo para cada término su índice en el vocabulario del vectorizador TF-IDF
# uso vocabulary_.get(p, None) para devolver None cuando el término no está presente en el vocabulario (evito KeyError)
# el resultado es una lista de índices (o None) con la misma longitud que 'palabras'
indice_de_palabras = [Tfidf_vectorizer.vocabulary_.get(p, None) for p in palabras]

In [13]:
# obtengo la lista completa de nombres de características (palabras del vocabulario) generada por el vectorizador TF-IDF
# get_feature_names_out() devuelve un array con todas las palabras asociadas a las columnas de la matriz TF-IDF
feature_names = Tfidf_vectorizer.get_feature_names_out()

In [15]:
# itero sobre pares (índice_de_palabra, palabra) alineados con zip para procesar cada término seleccionado
for idx, palabra in zip(indice_de_palabras, palabras):
  # verifico que el término exista en el vocabulario del vectorizador (evito errores si no está)
  if palabra in Tfidf_vectorizer.vocabulary_:
    # separador visual para facilitar lectura en consola
    print("="*100)
    # muestro el término original y su índice en el vocabulario
    print(f"\nPalabra original: {palabra} (índice {idx})")
    # calculo la similitud coseno entre el vector del término actual (fila idx de X_terms)
    # y todos los vectores de términos; uso reshape(1, -1) para convertirlo en una fila 2D compatible
    # el resultado 'cossim_words' es un vector con la similitud del término con cada término del vocabulario
    cossim_words = cosine_similarity(X_terms[idx].reshape(1, -1), X_terms)[0]

    # ordeno los índices por similaridad descendente y omito el primero (es el mismo término)
    # [1:6] devuelve los 5 términos más similares distintos del término actual
    mostsim_words = np.argsort(cossim_words)[::-1][1:6]

    # aviso que voy a listar las 5 palabras más similares
    print("5 palabras más similares:")
    # itero simultáneamente sobre los índices más similares y sus valores de similaridad
    for i, similaridad in zip(mostsim_words, cossim_words[mostsim_words]):
        # muestro el índice, la palabra correspondiente (mediante feature_names) y la similaridad formateada
        print(f"indice {i}, Palabra: {feature_names[i]}, similaridad: {similaridad:.4f}")
  # si el término no está en el vocabulario, imprimo un aviso claro
  else: print(f"\nPalabra '{palabra}' no esta en vocabulario.")


Palabra original: computer (índice 28881)
5 palabras más similares:
indice 32323, Palabra: decwriter, similaridad: 0.1579
indice 45612, Palabra: harkens, similaridad: 0.1531
indice 32658, Palabra: deluged, similaridad: 0.1531
indice 82134, Palabra: shopper, similaridad: 0.1421
indice 32606, Palabra: delicate, similaridad: 0.1366

Palabra original: god (índice 43733)
5 palabras más similares:
indice 52019, Palabra: jesus, similaridad: 0.2806
indice 22649, Palabra: bible, similaridad: 0.2764
indice 27138, Palabra: christ, similaridad: 0.2668
indice 39317, Palabra: faith, similaridad: 0.2593
indice 38539, Palabra: existence, similaridad: 0.2589

Palabra original: car (índice 25717)
5 palabras más similares:
indice 25863, Palabra: cars, similaridad: 0.1898
indice 30471, Palabra: criterium, similaridad: 0.1732
indice 32086, Palabra: dealer, similaridad: 0.1732
indice 27504, Palabra: civic, similaridad: 0.1713
indice 69023, Palabra: owner, similaridad: 0.1644

Palabra original: space (índic

Para 'computer', las asociaciones con términos como 'decwriter' (0.1579) y 'deluged' (0.1531) reflejan contextos técnicos de hardware y sobrecarga informativa, coherentes con categorías computacionales. 'God' muestra fuertes vínculos religiosos con 'jesus' (0.2806), 'bible' (0.2764) y 'christ' (0.2668), validando clusters temáticos en debates teológicos. 'Car' se relaciona lógicamente con 'cars' (0.1898), 'dealer' (0.1732) y 'civic' (0.1713), capturando temas automovilísticos. 'Space' destaca conexiones espaciales con 'nasa' (0.3279), 'shuttle' (0.2902) y 'seds' (0.2849), alineadas con exploración científica. Finalmente, 'politics' se asocia con 'iftccu' (0.3048), 'bmwmoa' (0.2606) y 'fascism' (0.2482), sugiriendo ideologías y grupos, aunque algunos acrónimos indican especificidad del dataset. En general, esta aproximación demuestra cómo TF-IDF en matrices transpuestas revela co-ocurrencias semánticas.


**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [16]:
# inicializo variables para guardar la mejor puntuación F1, el mejor modelo y los mejores parámetros del vectorizador
mejor_f1 = 0
mejor_modelo = None
mejor_vect_params = None

# defino una lista de configuraciones a probar para TfidfVectorizer
# cada diccionario contiene: max_df (umbral para eliminar términos muy frecuentes),
# min_df (mínimo de documentos en los que aparece un término para conservarlo) y ngram_range (n-gramas a considerar)
configuracion_vector = [
    {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 1)},  # configuración base: unigramas, conserva términos raros
    {'max_df': 0.8,  'min_df': 2, 'ngram_range': (1, 2)},  # incluir bigramas y filtrar términos que aparecen <2 docs
    {'max_df': 0.9,  'min_df': 3, 'ngram_range': (1, 1)},  # unigramas con min_df más alto para eliminar raros
    {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 3)}   # permitir hasta trigramas (mayor dimensionalidad)
]

# preparo la lista de modelos Naive Bayes a evaluar con distintos valores de suavizado (alpha)
# MultinomialNB suele funcionar bien con conteos/TF-IDF; ComplementNB es robusto ante desbalance
modelos = [
    MultinomialNB(alpha=1.0),  # suavizado Laplace por defecto
    MultinomialNB(alpha=0.5),  # menos suavizado (puede ajustar sesgo/varianza)
    ComplementNB(alpha=1.0),   # variante para correcciones en clases desbalanceadas
    ComplementNB(alpha=0.5)    # misma variante con menor suavizado
]

# itero sobre cada configuración del vectorizador
for vect_params in configuracion_vector:
    # instancio el TfidfVectorizer con los parámetros actuales
    tfidfvect_opt = TfidfVectorizer(**vect_params)
    # ajusto el vectorizador con los textos de entrenamiento y transformo entrenamiento
    X_train_opt = tfidfvect_opt.fit_transform(train_group.data)
    # transformo el conjunto de prueba con el mismo vectorizador (sin volver a ajustar)
    X_test_opt = tfidfvect_opt.transform(test_group.data)

    # itero sobre cada modelo definido
    for modelo in modelos:
        # ajusto el modelo con la matriz TF-IDF optimizada y las etiquetas de entrenamiento
        modelo.fit(X_train_opt, y_train)
        # predecir etiquetas sobre el conjunto de prueba
        y_pred = modelo.predict(X_test_opt)
        # calculo la métrica F1 macro para evaluar el desempeño promedio entre clases
        f1 = f1_score(y_test, y_pred, average='macro')
        # imprimo la configuración y el F1 obtenido para monitorizar resultados
        print(f"Config vect: {vect_params}, Model: {type(modelo).__name__} (alpha={modelo.alpha}), F1-macro: {f1:.4f}")

        # si la F1 actual supera la mejor encontrada, actualizo las variables que guardan el mejor resultado
        if f1 > mejor_f1:
            mejor_f1 = f1
            mejor_modelo = modelo
            mejor_vect_params = vect_params

# una vez probadas todas las combinaciones, muestro la mejor puntuación y sus parámetros asociados
print(f"\nMejor F1-macro: {mejor_f1:.4f}")
print("Mejores parametros para vectorizador :", mejor_vect_params)
print("Mejor modelo:", mejor_modelo)

Config vect: {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 1)}, Model: MultinomialNB (alpha=1.0), F1-macro: 0.5854
Config vect: {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 1)}, Model: MultinomialNB (alpha=0.5), F1-macro: 0.6153
Config vect: {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 1)}, Model: ComplementNB (alpha=1.0), F1-macro: 0.6930
Config vect: {'max_df': 0.95, 'min_df': 1, 'ngram_range': (1, 1)}, Model: ComplementNB (alpha=0.5), F1-macro: 0.6961
Config vect: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2)}, Model: MultinomialNB (alpha=1.0), F1-macro: 0.5703
Config vect: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2)}, Model: MultinomialNB (alpha=0.5), F1-macro: 0.5988
Config vect: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2)}, Model: ComplementNB (alpha=1.0), F1-macro: 0.6878
Config vect: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2)}, Model: ComplementNB (alpha=0.5), F1-macro: 0.6967
Config vect: {'max_df': 0.9, 'min_df': 3, 'ngram_range': (1, 1)}

Probe cuatro configuraciones de vectorización y cuatro variantes de modelos, lo cual resulta en 16 combinaciones, los resultados muestran que ComplementNB supera consistentemente a MultinomialNB, especialmente con alpha=0.5, al manejar mejor desbalances de clases, la mejor configuración fue max_df=0.8, min_df=2 y ngram_range=(1,2) con ComplementNB(alpha=0.5), alcanzando un F1-macro de 0.6967, mejorando sobre la base (0.5854) gracias a la inclusión de bigramas para capturar contextos y filtrado de términos raros/frecuentes para reducir ruido, por el contrario las configuraciones con trigramas empeoraron el desempeño por mayor dimensionalidad y posible sobreajuste, esto tambien demuestra la importancia de tuning de los hiperparámetros en tareas de clasificación de texto, donde ComplementNB es robusto para datasets multiclasse como este, sugiriendo potenciales mejoras con técnicas adicionales como stemming o cross-validation para mayor generalización.